In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from tools import bootstrap_ci
from joblib import Parallel, delayed
import h5py
from tqdm.notebook import tqdm,trange
sns.set_context("poster")

In [ ]:
hdf5_filename= "sim_results.h5"

In [ ]:
prol_data = pd.read_csv("responses.csv")
prol_data["advice"]=prol_data["advice"].astype(float)

In [ ]:
alg_styles = (
    ("exp4", "C2", "EXP4", '^'),
    ("random_expert", "C4", "random expert", 'D'),
    ("WMV", "C1", "WMV", 'v'),
    ("expert_0", "C7", "best expert", '>'),
    ("MetaCMAB", "C0", "MetaCMAB", '<'),
    ("ExpertiseTree", "black", "ExpertiseTree", 'p'),
)


def get_all_results(k, subset_size, treatment=None):

    if k == "random_expert":
        results = []
        for i in range(subset_size):
            results.append(get_all_results(f"expert_{i}",subset_size,treatment))
        return np.mean(results,axis=0)
    with h5py.File(hdf5_filename, 'r') as hf:
        if f"{subset_size} {0} {k}" not in hf:
            return None
        results = []
        for tr in range(5):
            data = hf[f"{subset_size} {tr} {k}"]
            if tr == treatment:
                return np.array(data)

            results.extend(np.array(data).reshape((-1, 16)))

        return np.array(results)


def print_summary(SUBSET_SIZE):
    print("algorithm"+" "*max(1, 35-len("algorithm")), "  mean", "  final")
    print("----------------------------------------------")
    with h5py.File(hdf5_filename, 'r') as hf:

        algorithms = set([" ".join(key.split()[2:])
                         for key in hf.keys() if key.split()[0] == str(SUBSET_SIZE)])

        for k in sorted(algorithms):
            try:
                all_results = get_all_results(k, SUBSET_SIZE)
                reward_mean = np.mean(all_results)
                final_reward_mean = np.mean(all_results[..., -1:])

                print(k+" "*max(1, 35-len(k)), f"{reward_mean:.4f}", f"{final_reward_mean:.4f}")
            except:
                print("skipping", k)


def plot_subset_figures(SUBSET_SIZE):
    os.makedirs("figures", exist_ok=True)

    def transform(a):
        ref_key = "expert_0"
        if mode == "instantaneous regret":
            return np.array(get_all_results(ref_key, SUBSET_SIZE))-a
        if mode == "regret":
            a = np.array(get_all_results(ref_key, SUBSET_SIZE))-a
            return np.cumsum(a, axis=1)
        if mode == "average":
            return np.cumsum(a,axis=1)/np.arange(1,len(a[0])+1)
      
        return a

    def plot_k_c(alg_key, alg_color, alg_label, alg_marker):
        metric = transform(np.array(get_all_results(alg_key, SUBSET_SIZE)))
        mean = metric.mean(axis=0)

        lower_bound, upper_bound = np.array(
            [bootstrap_ci(metric[:, i],n_bootstraps=1000) for i in range(T)]).T

        plt.plot(np.arange(len(mean))*3, (mean), color=alg_color, label=alg_label,
                 linestyle="--" if alg_label not in ("MetaCMAB", "ExpertiseTree") else "-", marker=alg_marker,
                 linewidth=4, markersize=8 if alg_marker == 'D' else 10)
        plt.fill_between(x=np.arange(len(mean))*3, y1=lower_bound,
                         y2=upper_bound, color=alg_color, alpha=0.2)

    for mode in ( "instantaneous regret",):

        for alg_key, alg_color, alg_label, alg_marker in alg_styles:
            if get_all_results(alg_key, SUBSET_SIZE) is None:
                continue

            plot_k_c(alg_key, alg_color, alg_label, alg_marker)

        plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))

        plt.ylabel(mode.replace("reward", "accuracy"))
        plt.xlabel("#headlines")

        if mode in ("reward", "average"):
            plt.yticks([0.5, 0.6, .7, .8, .9])

        if mode == "instantaneous regret":
            plt.ylim(-0.12, 0.3)
            plt.yticks([-0.1, 0, .1, .2, .3])

        plt.gca().set_box_aspect(1)
        sns.despine(offset=0,)

        plt.savefig(f"figures/{mode}_{SUBSET_SIZE}.pdf", bbox_inches="tight")
        plt.savefig(f"figures/{mode}_{SUBSET_SIZE}.svg", bbox_inches="tight")
        plt.show()

In [ ]:
from agent import *
from policy import *


class GreedyExpert():
    def __init__(self, expert_index, performance_index):
        self.expert_index = expert_index
        self.performance_index = performance_index

    def choose(self, advice):
        play_advice = advice["advice"][self.expert_index]
        probs = greedy_choice(play_advice)
        return np.searchsorted(np.cumsum(probs), np.random.rand(1))[0]

    def update(self, rewards, action):
        pass

    def reset(self):
        pass


def run_simulation(
    advice,
    outcomes,
    contexts,
    agent,
    seed,
    subset_size,
    n_simulations=100
):
    np.random.seed(seed)
    run_seeds = np.random.randint(0, 1000000, size=n_simulations)
    n_trials, n_arms, n_experts = advice.shape

    np.random.seed(seed)

    assert 0 < subset_size <= n_experts

    simulation_subsets = [np.random.choice(
        n_experts, size=subset_size, replace=False) for _ in range(n_simulations)]

    rewards = []

    for run_seed, subset in tqdm(zip(run_seeds, simulation_subsets), total=n_simulations, desc="inner", disable=True):

        np.random.seed(run_seed)
        rewards.append([])

        t_order = np.random.choice(n_trials, size=n_trials, replace=False)

        sim_advice = advice[t_order][..., subset]
        sim_outcomes = outcomes[t_order][..., subset]
        sim_contexts = contexts[t_order]

        # assign appropriate expert
        if type(agent) == GreedyExpert:
            expert_votes = greedy_choice(sim_advice, axis=1)
            expert_rewards = (expert_votes * sim_outcomes).sum(axis=1)
            agent.expert_index = np.argsort(-expert_rewards.sum(axis=0))[
                agent.performance_index]

        agent.reset()
        for t in range(n_trials):
            np.random.seed(run_seed + t)

            trial_rewards = sim_outcomes[t][..., 0]

            choice = agent.choose(
                {"advice": sim_advice[t].T, "context": sim_contexts[t]})
            agent.update(trial_rewards, choice)

            rewards[-1].append(trial_rewards[choice])

    return rewards

In [ ]:
subset_sizes = np.arange(2,37,2).tolist()

In [ ]:
data_by_treatment = [prol_data.query("treatment == @treatment").sort_values(
    by=["trial", "arm", "expert_id"]) for treatment in range(5)]

SEED = 1234
n_simulations = 1000
simulations_per_process = 25
np.random.seed(SEED)
simulation_seeds = np.random.randint(2**16-1,size=np.ceil(n_simulations / simulations_per_process).astype(int))

for sim_n_experts in tqdm(subset_sizes,desc="subset sizes"):
    for treatment in trange(5, desc="treatment", disable=False):

        treatment_data = data_by_treatment[treatment].copy()
        N_ARMS = treatment_data.arm.nunique()
        N_EXPERTS = treatment_data.expert_id.nunique()
        T = treatment_data.trial.nunique()

        advice, outcomes = treatment_data[[
            "advice", "genuine",
        ]].values.T.reshape((-1, T, N_ARMS, N_EXPERTS))

        contexts = np.rollaxis(treatment_data[[
            "outcome:white", "outcome:male", "outcome:young"
        ]].abs().values.T.reshape((-1, T, N_ARMS, N_EXPERTS))[..., 0], 0, 3)

        aggregators = {
            "WMV": Collective(N_ARMS, GreedyPolicy(), sim_n_experts),
            "exp4": Exp4(N_ARMS, Exp3Policy(), sim_n_experts, gamma=np.sqrt(0.5 * np.log(sim_n_experts + 1) / (N_ARMS * T))),
            "MetaCMAB":  MetaCMAB(N_ARMS, GreedyPolicy(), sim_n_experts ),
            "ExpertiseTree": ExpertiseTree(N_ARMS, GreedyPolicy(), sim_n_experts),
        }
        for i in range(sim_n_experts):
            aggregators[f'expert_{i}'] = GreedyExpert(None, i)

        bar = tqdm(aggregators.items(), total=len(
            aggregators), disable=False, desc="aggregators")
        for k, agent in bar:
            dataset_key = f"{sim_n_experts} {treatment} {k}"
            bar.set_description(k)
            
            if  os.path.exists(hdf5_filename):
                with h5py.File(hdf5_filename, 'r') as hf:
                    if dataset_key in hf:
                        continue

            results = Parallel(n_jobs=8)(delayed(run_simulation)(
                advice, outcomes, contexts, agent, seed, sim_n_experts, 
                simulations_per_process if (i + 1) * simulations_per_process <= n_simulations else n_simulations - i * simulations_per_process) for i,seed in enumerate(simulation_seeds))
            results = np.array(results).reshape((-1, T))

            with h5py.File(hdf5_filename, 'a') as hf:
                if dataset_key in hf:
                    del hf[dataset_key]
                hf.create_dataset(dataset_key, data=results)

    print_summary(sim_n_experts)
    plot_subset_figures(sim_n_experts)

In [ ]:

fig,axes = plt.subplots(2,5)
fig.set_size_inches(18.5, 6)
row_length=5
for n,subset_size in enumerate([2]+np.arange(4,37,4).tolist()):
    row_labels = ('ExpertiseTree','MetaCMAB',"WMV","exp4","random_expert")[::-1]
    proper_labels = ['ExpertiseTree',"MetaCMAB","WMV","EXP4","random member"][::-1]
    

    reference_results =  np.array([get_all_results(f'expert_{i}',subset_size)[...,:].mean(axis=-1) for i in range(subset_size)])
    alg_results = [get_all_results(k,subset_size)[...,:].mean(axis=-1) for k in row_labels]

    percent_scores=np.array([(((alg_results[j]>=reference_results)).mean(axis=1) + ((alg_results[j]>reference_results)).mean(axis=1))/2 for j in range(len(row_labels))])
    print(subset_size,percent_scores[-2:,...,:2])
    j=n%row_length
    i=n//row_length
    im = axes[i,j].imshow(percent_scores,aspect=.12,extent=[0,1,0,len(row_labels)],cmap="seismic",vmin=0,vmax=1)

    if j==0:
        axes[i,j].set_yticks(np.arange(len(row_labels))+.5, proper_labels[::-1])
        axes[i,j].set_ylabel("algorithm")
    else:
        axes[i,j].set_yticks(np.arange(len(row_labels))+.5, [""]*len(row_labels))
    if i==0:
        
        axes[i,j].set_xticklabels([])
    else:
        axes[i,j].set_xticklabels(['0','0.5','1'])

    if i==1:
        axes[i,j].set_xlabel("member quantile")
    axes[i,j].set_title(f"N={subset_size}")
    
cbar_ax = fig.add_axes([0.925, 0.125, 0.01, 0.7])
cbar = fig.colorbar(im, cax=cbar_ax)

cbar.ax.get_yaxis().labelpad = 25
cbar.set_label('win ratio', rotation=270)
plt.savefig(f"figures/quantile_heatmap.pdf",bbox_inches="tight")
plt.savefig(f"figures/quantile_heatmap.svg",bbox_inches="tight")
plt.show()


In [ ]:

for alg_key, alg_color, alg_label, alg_marker in alg_styles:
    try:
        results = [get_all_results(alg_key,subset_size).mean(axis=-1) for subset_size in subset_sizes]
    except:
        continue
    means = [np.mean(result) for result in results]
    print(alg_key,np.mean(means))
    cis = np.array([bootstrap_ci(result) for result in results])
    
    plt.plot(subset_sizes,means,  color=alg_color, label=alg_label, 
             linestyle=":" if 'b' in alg_key else "--" if alg_label not in ("MetaCMAB", "ExpertiseTree") else "-", 
             linewidth=4, markersize=8 if alg_marker == 'D' else 10,
             marker=alg_marker)

    plt.fill_between(subset_sizes, cis[:,0],cis[:,1], color=alg_color, alpha=.2)


plt.xlabel("group size")
plt.ylabel("accuracy")
sns.despine()
plt.gca().set_box_aspect(1)
sns.despine(offset=0,)

plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.savefig("figures/average_reward.pdf",bbox_inches="tight") 
plt.savefig("figures/average_reward.svg",bbox_inches="tight") 

In [ ]:

for alg_key, alg_color, alg_label, alg_marker in alg_styles:
    
    try:
        results = [get_all_results("expert_0",subset_size)[:,-1]-get_all_results(alg_key,subset_size)[:,-1] for subset_size in subset_sizes]
    except:
        continue
    means = [np.mean(result) for result in results]
    print(alg_key,np.mean(means))
    cis = np.array([bootstrap_ci(result) for result in results])
    plt.plot(subset_sizes,means, color=alg_color, label=alg_label, 
             linestyle=":" if 'b' in alg_key else "--" if alg_label not in ("MetaCMAB", "ExpertiseTree") else "-", 
             linewidth=4, markersize=8 if alg_marker == 'D' else 10,
             marker=alg_marker)

    plt.fill_between(subset_sizes, cis[:,0],cis[:,1], color=alg_color, alpha=.2)


plt.xlabel("group size")
plt.ylabel("terminal regret")
plt.gca().set_box_aspect(1)
sns.despine(offset=0,)

plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.savefig("figures/final_reward.pdf",bbox_inches="tight") 
plt.savefig("figures/final_reward.svg",bbox_inches="tight") 